# Keyphrase similality with spaCy

Date: 2024/06/16

In [1]:
import spacy
import textacy
from textacy import extract

In [2]:
# Text generated with ChatGPT
text = open("beatles.txt").read()

In [3]:
nlp = spacy.load("ja_core_news_lg")
doc = nlp(text)

/Users/shiny/Documents/GitHub/learning-langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sents = [sent.text.replace('・', ' ') for sent in doc.sents]

In [5]:
en = textacy.load_spacy_lang("ja_core_news_lg")

In [6]:
import re
kps = []

def normalize(x):
    if re.search(r'[ぁ-ん]+|[ァ-ヴー]+|[一-龠]+', x):
        return x.replace(' ', '')
    else:
        return x.lower()
        
for sent in sents:
    doc = textacy.make_spacy_doc(sent, lang=en)
    kps_textrank = [normalize(kps) for kps, _ in extract.keyterms.textrank(doc, normalize="lemma", topn=10)]
    #print(kps_textrank)
    kps.extend(kps_textrank)

In [7]:
kps

['ロックバンド',
 '年代',
 'イギリス',
 '人気',
 '世界',
 'ビートルズ',
 'ジョンレノン',
 'リンゴスター',
 'ポールマッカートニー',
 'ジョージハリスン',
 'ringo starr',
 'paul mccartney',
 'john lennon',
 'george harrison',
 '人',
 'メンバー',
 '音楽史',
 '多様性',
 '革新性',
 'ハードロック',
 'サイケデリックロック',
 '大きな影響',
 '多岐',
 'ポップ',
 '結成',
 '活動',
 'ビートルズ',
 '初期',
 'リヴァプール',
 'ピートベスト',
 'レノン',
 'メンバー',
 'ドラマー',
 'マッカートニー',
 'ハリスン',
 '当初',
 '演奏活動',
 'ハンブルク',
 '過酷',
 'スキル',
 'クラブ',
 '結束力',
 '期間',
 '音楽',
 'バンド',
 '成長',
 'リンゴスター',
 'ジョージマーティン',
 '年',
 'バンド',
 'ベスト',
 '代わり',
 '勧め',
 'プロデューサー',
 'ラヴミードゥ',
 'love me do',
 'デビューシングル',
 'ブレイクスルー',
 '始まり',
 '年',
 'ビートルズ',
 '月',
 '成功',
 'フロムミートゥユー',
 'from me to you',
 'プリーズプリーズミー',
 'please please me',
 'シーラヴズユー',
 'she loves you',
 'イギリス国内',
 'ヒット曲',
 '年',
 '爆発',
 'エドサリヴァンショー',
 'ビートルマニア',
 '進出',
 'アメリカ',
 '出演',
 '年',
 'ビートルズ',
 'アメリカ国内',
 '一大センセーション',
 'ビートルマニア',
 'パフォーマンス',
 '現象',
 'i want to hold your hand',
 'ビルボードチャート',
 'トップスター',
 '抱く',
 'シングル',
 'アメリカ',
 '位',
 '座',
 '大きな進化',
 '年',
 '音楽',
 'ビートルズ',
 '高い評価',
 '音

In [8]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb import Documents, EmbeddingFunction, Embeddings

class SpacyEmbedder(EmbeddingFunction):

    def __init__(self):
        self.nlp_ja = spacy.load("ja_core_news_lg")
        self.nlp_en = spacy.load("en_core_web_lg")
        
    def __call__(self, texts):
        embeddings = []
        for text in texts:
            if re.search(r'[ぁ-ん]+|[ァ-ヴー]+|[一-龠]+', text):
                embeddings.append(list(self.nlp_ja(text).vector.astype(float)))
            else:
                embeddings.append(list(self.nlp_en(text).vector.astype(float)))
        return embeddings

#chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="./embeddings_spacy.db")

In [9]:
#collection = chroma_client.create_collection(name="collection-beatles3", embedding_function=multi)
collection = chroma_client.get_or_create_collection(name="collection-beatles", embedding_function=SpacyEmbedder())

In [10]:
ids = [str(i) for i in range(len(kps))]
metadatas = [dict(word_num=i) for i in range(len(kps))]

In [11]:
collection.add(
    documents=kps,
    ids=ids,
    metadatas=metadatas
)

Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [12]:
collection.get("1")

{'ids': ['1'],
 'embeddings': None,
 'metadatas': [{'translation': 'generations', 'word_num': '1'}],
 'documents': ['年代'],
 'uris': None,
 'data': None}

In [13]:
collection.update(
    ids=["1"],
    metadatas=[dict(word_num=str(1), translation="generations")]
)

In [14]:
collection.get("1")

{'ids': ['1'],
 'embeddings': None,
 'metadatas': [{'translation': 'generations', 'word_num': '1'}],
 'documents': ['年代'],
 'uris': None,
 'data': None}

In [15]:
QUERY_TEXTS = ["ジョンレノン", "ジョン", "ポール", "ジョージ", "リンゴ", "影響力", "サイケデリック", "インド", "移住", "成功",
               "レジェンド", "レガシー", "Sgt pepper's lonely hearts club band", "Popular"]

In [16]:
results = collection.query(
    query_texts=QUERY_TEXTS,
    n_results=5 # how many results to retur
)

results

{'ids': [['6', '202', '30', '112', '300'],
  ['47', '9', '201', '7', '199'],
  ['8', '200', '47', '9', '201'],
  ['47', '9', '201', '69', '199'],
  ['65', '135', '288', '255', '222'],
  ['194', '280', '298', '140', '239'],
  ['111', '20', '209', '23', '0'],
  ['138', '69', '80', '76', '90'],
  ['80', '69', '14', '54', '67'],
  ['62', '204', '195', '123', '80'],
  ['7', '46', '199', '108', '167'],
  ['210', '74', '82', '167', '46'],
  ['87', '237', '274', '72', '91'],
  ['32', '19', '311', '167', '0']],
 'distances': [[0.0,
   0.0,
   7.017339706420898,
   7.017339706420898,
   12.49207575020257],
  [5.489184856414795,
   5.489184856414795,
   5.489184856414795,
   8.190232276916504,
   8.190232276916504],
  [3.960376501083374,
   3.960376501083374,
   10.48888874053955,
   10.48888874053955,
   10.48888874053955],
  [2.861128091812134,
   2.861128091812134,
   2.861128091812134,
   10.627386093139648,
   10.774742126464844],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 3.084672689437

In [17]:
ids = results["ids"]
distances = results["distances"]
documents = results["documents"]
metadatas = results["metadatas"]

THRESHOLD = 10000

for idx, dist in enumerate(results["distances"]):
    for i, d in enumerate(dist):
        if d <= THRESHOLD:
            print(QUERY_TEXTS[idx], ids[idx][i], documents[idx][i], f"{distances[idx][i]:.3f}", f"{metadatas[idx][i]}")

ジョンレノン 6 ジョンレノン 0.000 {'word_num': 6}
ジョンレノン 202 ジョンレノン 0.000 {'word_num': 202}
ジョンレノン 30 レノン 7.017 {'word_num': 30}
ジョンレノン 112 レノン 7.017 {'word_num': 112}
ジョンレノン 300 ビートルズ 12.492 {'word_num': 300}
ジョン 47 ジョージマーティン 5.489 {'word_num': 47}
ジョン 9 ジョージハリスン 5.489 {'word_num': 9}
ジョン 201 ジョージハリスン 5.489 {'word_num': 201}
ジョン 7 リンゴスター 8.190 {'word_num': 7}
ジョン 199 リンゴスター 8.190 {'word_num': 199}
ポール 8 ポールマッカートニー 3.960 {'word_num': 8}
ポール 200 ポールマッカートニー 3.960 {'word_num': 200}
ポール 47 ジョージマーティン 10.489 {'word_num': 47}
ポール 9 ジョージハリスン 10.489 {'word_num': 9}
ポール 201 ジョージハリスン 10.489 {'word_num': 201}
ジョージ 47 ジョージマーティン 2.861 {'word_num': 47}
ジョージ 9 ジョージハリスン 2.861 {'word_num': 9}
ジョージ 201 ジョージハリスン 2.861 {'word_num': 201}
ジョージ 69 イギリス国内 10.627 {'word_num': 69}
ジョージ 199 リンゴスター 10.775 {'word_num': 199}
リンゴ 65 プリーズプリーズミー 0.000 {'word_num': 65}
リンゴ 135 マハリシマヘーシュヨーギー 0.000 {'word_num': 135}
リンゴ 288 ザビートルズ 0.000 {'word_num': 288}
リンゴ 255 ザビートルズ 0.000 {'word_num': 255}
リンゴ 222 アーティスト 0.000 {'word_num': 222}
影響

## Conclusion

The model is clever.